# Perfect forwarding

In generic programming, the problem of perfect forwarding arises when a function template `f(T x)` working on an entity of type `T` invokes another function template `g` from within its body, using the same entity `x`. We want to be able to _forward_ `x` as it was to `g`. If `x` was an L-value reference, we want `g` to be invoked using an L-value reference. If the input is an R-value reference, we want `g` to be called accordingly.  

In [ ]:
#pragma cling add_include_path("/p/project/training2444/local/include")
#include <Vbose.hh>

In [ ]:
Vbose l{"0000"};

In [ ]:
namespace v0{
template <class T>
void g(T x) {
    std::cout << "Inside function g\n";
    Vbose a{x};
    std::cout << "Leaving g()\n";
}
template <class T>
void f(T x) {
    std::cout << "Inside f(), calling g() ...\n";
    g(x);
    std::cout << "Leaving f()\n";
}
}

A naive implementation as above leads to some unnecessary copies, which can be easily avoided. Here we call `v0::f` using a pre-existing name `l`, i.e., an L-value reference.

In [ ]:
v0::f(l);

Now we call it using an unnamed temporary, i.e., an R-value reference.

In [ ]:
v0::f(Vbose{"1111"});

Of course, you realise that we have been passing parameters to the functions `f` and `g` as if they were ordinary value parameters. Although `T` is inferred by the compiler, as written above, `T` is deduced as a value type, not as a reference. What if we changed the inputs to constant references ...

In [ ]:
namespace v1{
template <class T>
void g(const T& x) {
    std::cout << "Inside function g\n";
    Vbose a{x};
    std::cout << "Leaving g()\n";
}
template <class T>
void f(const T& x) {
    std::cout << "Inside f(), calling g() ...\n";
    g(x);
    std::cout << "Leaving f()\n";
}
}

In [ ]:
v1::f(l);

How wonderful! Just one copy constructor called when we created the `Vbose` object inside `g`. No spurious copies. But...

In [ ]:
v1::f(Vbose{"2222"});

Here we call `f` with a temporary, an R-value reference. The class `Vbose` has a move constructor. It would have been nice if `g` somehow got the information from `f`. We want the following: 

    - `g` should be called with an L-value reference if the input to `f` is one.
    - `g` should be called with an R-value reference if the input to `f` is one.

By making our definitions explicitly `const T&` above we have lost this flexibility. How should we make our template result in different types of references depending on the different types of references in the input?

Introducing _forwarding references_...

In [ ]:
namespace v2{
template <class T>
void g(T&& x) {
    std::cout << "Inside function g\n";
    Vbose a{std::forward<T>(x)};
    std::cout << "Leaving g()\n";
}
}

In the above definition, the input to `g` is *not* an R-value reference. A template parameter written in terms of a seemingly R-value reference to a placeholder type `T` has special rules of deduction. 

  - if `x` is an L-value reference, `T` is inferred to be an L-value reference, so that the function is translated as if you wrote `void g((sometype&) && x) ...`. The reference compression rules of C++ then make this simply `void g(sometype& x)...`.
  - If `x` is an R-value reference, `T` is inferred to be the ordinary value type, so that the function is translated as if you wrote `void g(sometype && x) ...`.

    In both cases, inside the function we have the full reference qualified type of `x` in  `T&&`. However, since `x` itself is a name, if we simply constructed `a` as `a{x}`, we would still be calling the copy constructor of `Vbose`. We want to cast the symbol `x` into the appropriate reference qualified type, something like: `static_cast<T&&>(x)`. A pretty way to write that is `std::forward<T>(x)`, as done above. Let's see what it does...

In [ ]:
v2::g(l);

As expected, the `Vbose` constructor called inside `g` is a copy constructor, since we called `g` with an L-value reference. More importantly...

In [ ]:
v2::g(Vbose{"3333"});

when we call `g` with a temporary object, the same function `g` calls the move constructor of `Vbose`, because it forwards the "temporariness" information of `x` to the constructor call. Using the same idea on `f` we get...

In [ ]:
namespace v2{
template <class T>
void f(T&& x) {
    std::cout << "Inside f(), calling g() ...\n";
    g(std::forward<T>(x));
    std::cout << "Leaving f()\n";
}
}

In [ ]:
v2::f(l);

In [ ]:
v2::f(Vbose{"4444"});

Only the essential constructor calls remain now, as the reference type information is now correctly forwarded by the outer layers. Let's now examine this for another situation where the impact is more drastic! 

In [ ]:
template <class T>
struct Entity0 {
  Entity0(T x, T y) : l{x}, r{y} {}

  T l, r;
};

In [ ]:
Vbose lvl{"0000"};

In [ ]:
{
    Entity0<Vbose> e{lvl, Vbose{"1111"}};
}

This is expected, because we are using ordinary types instead of forwarding references as the constructor inputs. Using the lessons learned above, we can write...

In [ ]:
template <class T>
struct Entity1 {
  template <class U, class V>
  Entity1(U&& x, V&& y) : l{std::forward<U>(x)}, r{std::forward<V>(y)} {}

  T l, r;
};

In [ ]:
{
    Entity1<Vbose> e{lvl, Vbose{"2222"}};
}

Notice how the move constructor is used when the input allows it. Change the above cell by making the first input a temporary, both or none temporaries etc. You will see that the above actually behaves as if we had written the following 4 constructors!
```
struct Entity2 {
  Entity2(const Vbose& x, const Vbose& y) : l{x}, r{y} {}
  Entity2(const Vbose& x, Vbose&& y) : l{x}, r{std::move(y)} {}
  Entity2(Vbose&& x, const Vbose& y) : l{std::move(x)}, r{y} {}
  Entity2(Vbose&& x, Vbose&& y) : l{std::move(x)}, r{std::move(y)} {}

  Vbose l, r;
};
```


To illustrate the usefulness of these forwarding references, and demonstrate their syntax when using variadic templates, we consider the following container type.

In [ ]:
template <class T> struct Container {
    template <class ...Args>
    void emplace(Args... args) {
        std::cout << "Entered the emplace function.\n";
        T obj{args...};
        std::cout << "Created contained type.\n";
    }
};

In [ ]:
{
    Container<Entity1<Vbose>> v;
    v.emplace(lvl, Vbose{"CCCC"});
}

Simply writing the emplace in terms of constant references is inadequate...

In [ ]:
template <class T> struct Container2 {
    template <class ...Args>
    void emplace(const Args&... args) {
        std::cout << "Entered the emplace function.\n";
        T obj{args...};
        std::cout << "Created contained type.\n";
    }
};

In [ ]:
{
    Container2<Entity1<Vbose>> v;
    v.emplace(lvl, Vbose{"CCCC"});
}

We need to _forward_ the inputs to emplace down to the object constructor, like so...

In [ ]:
template <class T> struct Container3 {
    template <class ...Args>
    void emplace(Args&&... args) {
        std::cout << "Entered the emplace function.\n";
        T obj{std::forward<Args>(args)...};
        std::cout << "Created contained type.\n";
    }
};

In [ ]:
{
    Container3<Entity1<Vbose>> v;
    v.emplace(lvl, Vbose{"CCCC"});
}